In [256]:
import pandas as pd

def IoTcode2vulBERT(csv_input, csv_output):
    df = pd.read_csv(csv_input)
    """ convert data to vulBERT format """
    if 'is_vul' in list(df.columns):
        df = df[['code', 'is_vul']]
        df = df.rename(columns={'is_vul':'label'})
        df['label'] = df.label.astype(int)

    df.to_csv(csv_output)
    return df

dataset = 'raspberryZip'
csv_input = f"../data/{dataset}_function.csv"
csv_vulbert =f"../data/{dataset}_function-VulBERT.csv"
# csv = '../VulBERTa/data/finetune/d2a/function/d2a_lbv1_function_dev.csv'

df = IoTcode2vulBERT(
    csv_input=csv_input,
    csv_output=csv_vulbert
    )

In [257]:
dataset = 'raspberryZip'
flaw = f"../data/{dataset}_statement.csv"
metrics = f"../data/{dataset}_function.csv"
binary = f"../data/{dataset}_method_binary.csv"

df_stat  = pd.read_csv(flaw)
df_fun = pd.read_csv(metrics)

print(f'Fun statistics: \n{df_fun.isVul.value_counts()}')
print(f'\nStat statistics: \n{df_stat.cwe.value_counts()}')

Fun statistics: 
0    11677
1      982
Name: isVul, dtype: int64

Stat statistics: 
benign              30397
CWE-120               403
CWE-119!/CWE-120      254
CWE-126               220
CWE-78                114
CWE-120, CWE-20        97
CWE-134                30
CWE-362                26
CWE-362/CWE-367!       20
CWE-732                11
CWE-457                 1
Name: cwe, dtype: int64


In [258]:
df_fun.drop_duplicates(subset=['filename', 'long_name', 'start_line', 'end_line', 'cwe'], keep='last').reset_index(drop=True)

,cyclomatic_complexity,nloc,token_count,name,long_name,start_line,end_line,filename,code,fun_name,content,isVul,cwe
0,2,5,46,BPF_CALL_2,"BPF_CALL_2( bpf_map_lookup_elem , struct bpf_m...",34,38,data/projects/linux-rpi-6.1.y/kernel/bpf/helpe...,"BPF_CALL_2( bpf_map_lookup_elem , struct bpf_m...",BPF_CALL_2,NaN,0,['Benign']
1,2,6,55,BPF_CALL_4,"BPF_CALL_4( bpf_map_update_elem , struct bpf_m...",49,54,data/projects/linux-rpi-6.1.y/kernel/bpf/helpe...,"BPF_CALL_4( bpf_map_update_elem , struct bpf_m...",BPF_CALL_4,NaN,0,['Benign']
2,2,5,42,BPF_CALL_2,"BPF_CALL_2( bpf_map_delete_elem , struct bpf_m...",67,71,data/projects/linux-rpi-6.1.y/kernel/bpf/helpe...,"BPF_CALL_2( bpf_map_delete_elem , struct bpf_m...",BPF_CALL_2,NaN,0,['Benign']
3,1,4,35,BPF_CALL_3,"BPF_CALL_3( bpf_map_push_elem , struct bpf_map...",82,85,data/projects/linux-rpi-6.1.y/kernel/bpf/helpe...,"BPF_CALL_3( bpf_map_push_elem , struct bpf_map...",BPF_CALL_3,NaN,0,['Benign']
4,1,4,29,BPF_CALL_2,"BPF_CALL_2( bpf_map_pop_elem , struct bpf_map ...",97,100,data/projects/linux-rpi-6.1.y/kernel/bpf/helpe...,"BPF_CALL_2( bpf_map_pop_elem , struct bpf_map ...",BPF_CALL_2,NaN,0,['Benign']
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12654,19,44,299,gdb_cmd_break,gdb_cmd_break( struct kgdb_state * ks),860,919,data/projects/linux-rpi-6.1.y/kernel/debug/gdb...,gdb_cmd_break( struct kgdb_state * ks){\n\t/*\...,gdb_cmd_break,"['\t\tstrcpy(remcom_out_buffer, ""OK"");']",1,['CWE-120']
12655,5,19,106,gdb_cmd_exception_pass,gdb_cmd_exception_pass( struct kgdb_state * ks),922,949,data/projects/linux-rpi-6.1.y/kernel/debug/gdb...,gdb_cmd_exception_pass( struct kgdb_state * ks...,gdb_cmd_exception_pass,NaN,0,['Benign']
12656,35,111,493,gdb_serial_stub,gdb_serial_stub( struct kgdb_state * ks),954,1090,data/projects/linux-rpi-6.1.y/kernel/debug/gdb...,gdb_serial_stub( struct kgdb_state * ks){\n\ti...,gdb_serial_stub,['\t\tunsigned char thref[BUF_THREAD_ID_SIZE];...,1,"['CWE-120', 'CWE-126', 'CWE-119!/CWE-120']"
12657,5,26,121,gdbstub_state,"gdbstub_state( struct kgdb_state * ks , char *...",1092,1118,data/projects/linux-rpi-6.1.y/kernel/debug/gdb...,"gdbstub_state( struct kgdb_state * ks , char *...",gdbstub_state,['\t\tgdbstub_use_prev_in_buf = strlen(remcom_...,1,['CWE-126']


In [259]:
df_fun.cwe.value_counts()

['Benign']                                             11677
['CWE-120']                                              234
['CWE-unknown']                                          160
['CWE-119!/CWE-120']                                     144
['CWE-126']                                              116
['CWE-78']                                                63
['CWE-120, CWE-20']                                       53
['CWE-457']                                               51
['CWE-120', 'CWE-119!/CWE-120']                           32
['CWE-120', 'CWE-126']                                    17
['CWE-126', 'CWE-119!/CWE-120']                           17
['CWE-120', 'CWE-126', 'CWE-119!/CWE-120']                12
['CWE-562']                                               11
['CWE-570']                                               11
['CWE-362']                                               11
['CWE-134']                                               10
[nan]                   

In [260]:
df_fun[df_fun.cwe=="['u', 'w', 'E', 'C', 'W', 'o', '-', 'n', 'k']"]

,cyclomatic_complexity,nloc,token_count,name,long_name,start_line,end_line,filename,code,fun_name,content,isVul,cwe


In [261]:
print(len(df_stat.columns)) 
df_stat = df_stat.dropna(axis=1, how='all')
print(len(df_stat.columns))
df_stat

18
12


,file,line,column,category,name,msg,note,cwe,context,helpuri,tool,severity
0,data/projects/linux-rpi-6.1.y/kernel/printk/pr...,1069,8.0,buffer,char,Statically-sized arrays can be improperly rest...,"Perform bounds checking, use functions that li...",CWE-119!/CWE-120,static char setup_text_buf[LOG_LINE_MAX] __ini...,https://cwe.mitre.org/data/definitions/119.html,FlawFinder,NaN
1,data/projects/linux-rpi-6.1.y/kernel/trace/tra...,1353,2.0,buffer,char,Statically-sized arrays can be improperly rest...,"Perform bounds checking, use functions that li...",CWE-119!/CWE-120,char num_buf[24]; /* Big enough to hold an add...,https://cwe.mitre.org/data/definitions/119.html,FlawFinder,NaN
2,data/projects/linux-rpi-6.1.y/kernel/trace/tra...,702,2.0,buffer,char,Statically-sized arrays can be improperly rest...,"Perform bounds checking, use functions that li...",CWE-119!/CWE-120,char buffer[USTRING_BUF_SIZE];,https://cwe.mitre.org/data/definitions/119.html,FlawFinder,NaN
3,data/projects/linux-rpi-6.1.y/kernel/printk/pr...,181,1.0,buffer,char,Statically-sized arrays can be improperly rest...,"Perform bounds checking, use functions that li...",CWE-119!/CWE-120,"char devkmsg_log_str[DEVKMSG_STR_MAX_SIZE] = ""...",https://cwe.mitre.org/data/definitions/119.html,FlawFinder,NaN
4,data/projects/linux-rpi-6.1.y/kernel/printk/pr...,186,2.0,buffer,char,Statically-sized arrays can be improperly rest...,"Perform bounds checking, use functions that li...",CWE-119!/CWE-120,char old_str[DEVKMSG_STR_MAX_SIZE];,https://cwe.mitre.org/data/definitions/119.html,FlawFinder,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
31568,data/projects/linux-rpi-6.1.y/kernel/events/co...,5309,NaN,NaN,NaN,NaN,NaN,benign,values[n++] = atomic64_read(&leader->lost_samp...,NaN,sampling,NaN
31569,data/projects/linux-rpi-6.1.y/kernel/events/co...,5243,NaN,NaN,NaN,NaN,NaN,benign,*enabled += event->total_time_enabled +,NaN,sampling,NaN
31570,data/projects/linux-rpi-6.1.y/kernel/events/co...,5232,NaN,NaN,NaN,NaN,NaN,benign,struct perf_event *child;,NaN,sampling,NaN
31571,data/projects/linux-rpi-6.1.y/kernel/events/co...,5326,NaN,NaN,NaN,NaN,NaN,benign,struct perf_event *leader = event->group_leade...,NaN,sampling,NaN


In [262]:
str(df_fun[df_fun.cwe=="['CWE-120', 'CWE-120, CWE-20']"]['filename'])

'8081    data/projects/linux-rpi-6.1.y/kernel/trace/rin...\nName: filename, dtype: object'

In [264]:
name = ['guru', 'bhandari']
print(len(name))
len(set(name)) and isinstance(name, list)

2


True